## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/학회/TextMining_WordEmbedding

/content/drive/MyDrive/학회/TextMining_WordEmbedding


In [3]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [4]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [5]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [6]:
# data 불러오기 (뉴스 기사 데이터)
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [7]:
corpus[:50]

['7개국 60명 연구 결과 \'네이처\' 발표…"암·선천성 기형 연구에 도움"',
 "실험동물로 알려진 '아프리카발톱개구리(Xenopus laevis)'의 유전체를 UNIST(울산과학기술원) 교수가 포함된 국제 공동연구진이 해독했다",
 "UNIST 권태준 생명과학부 교수(울산=연합뉴스) UNIST는 권태준 생명과학부 교수가 제1 저자로 참여하고 미국, 일본 등 총 7개 나라 60명이 함께한 공동연구진은 아프리카발톱개구리의 4만여 개 유전자를 염색체 수준으로 규명해 '네이처(Nature)'에 발표했다고 20일 밝혔다",
 '2016.10.20',
 '[UNIST 제공=연합뉴스]',
 '다양한 생물이 어떻게 진화했는지 풀어낼 수 있는 열쇠를 찾은 것이어서 향후 진화 연구에 도움이 될 것으로 보인다',
 "권태준 생명과학부 교수가 제1 저자로 참여하고 미국, 일본 등 총 7개 나라 60명이 함께한 공동연구진은 아프리카발톱개구리의 4만여 개 유전자를 염색체 수준으로 규명해 '네이처(Nature)'에 발표했다",
 '유전자 발현 조절이 쉽고 체외수정으로 한 번에 수백 개의 알을 얻을 수 있는 아프리카발톱개구리는 척추동물 유전자 연구에 널리 사용됐다',
 "2012년 노벨생리의학상을 수상한 존 고든(Sir John Gordon)이 체세포 핵치환 실험을 통해 '어른 세포가 다시 배아가 될 수 있다'는 걸 처음 보여준 실험에도 아프리카발톱개구리가 사용됐다",
 '그러나 정작 이 개구리의 유전체 해독은 다른 동물보다 어려웠다',
 '인간이나 다른 동물은 부모로부터 하나씩의 염색체 그룹을 물려받아 2개의 염색체 그룹(2배체)을 가지지만, 이 개구리는 부모에게서 두 개씩 염색체 그룹을 받아 4개 염색체 그룹(4배체)을 가지기 때문에 분석이 복잡했다',
 "공동연구진은 지난 2010년 해독된 '서양발톱개구리(Xenopus tropicalis)'를 이용해 분석을 시도했다",
 '인간처럼 2개 염색체 그룹을 가지는 서양발톱개구리를 아프리카발톱개구리와 비교해 부모로부터 물려받는 염색

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]

기사 데이터의 경우, 여러번 검수를 하기 때문에 오타가 있을 가능성이 낮고, 정보 전달 식의 문장이 많은 기사 특성상 특수 문자에 따라 의미가 달라지거나 하지 않기 때문에 Josa와 Punctuation은 제거했다.

In [8]:
okt = Okt()

In [9]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence, stop = ['Josa', 'Punctuation']):
    tokenized = []
    for i, j in okt.pos(sentence, stem = True, norm = True):
        if j in stop: # stop에 있는 게 품사가 되면, tokenize 패스하도록
            continue
        tokenized.append(i)
    return tokenized

In [14]:
i = 1
print(corpus[i])
print(CustomTokenization(corpus[i]))

실험동물로 알려진 '아프리카발톱개구리(Xenopus laevis)'의 유전체를 UNIST(울산과학기술원) 교수가 포함된 국제 공동연구진이 해독했다
['실험동물', '알려지다', '아프리카', '발톱개구리', 'Xenopus', 'laevis', '의', '유전체', 'UNIST', '울산', '과학기술원', '교수', '포함', '되다', '국제', '공', '동', '연', '구진', '해독', '하다']


In [10]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

# 2번

이제 word2vec 모델을 학습시켜 봅시다!


1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

데이터 양이 적어서 min_count =0 으로 해서 데이터 내 모든 단어에 대해 학습을 진행하도록 했다.

그리고 window 크기를 8로 늘려서 긴 문자에 대해서도 학습이 잘 진핻될 수 있도록 했다.

In [79]:
import numpy as np

len_sentence = []
for k in range(len(corpus)):
    len_sentence.append(len(corpus_tokenized[k]))
np.min(len_sentence)

1

In [116]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(size = 200, min_count = 0, window = 8) # 여기 설정하기

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 4123


In [117]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 30) # 이부분 값 넣기

(486512, 546210)

In [118]:
model_no_pretrained.wv.vocab.keys()

dict_keys(['7', '개국', '60', '명', '연구', '결과', '네이처', '발표', '암', '선천성', '기형', '도움', '실험동물', '알려지다', '아프리카', '발톱개구리', 'Xenopus', 'laevis', '의', '유전체', 'UNIST', '울산', '과학기술원', '교수', '포함', '되다', '국제', '공', '동', '연', '구진', '해독', '하다', '권태', '준', '생명', '과학부', '연합뉴스', '늘다', '제', '1', '저자', '참여', '미국', '일본', '등', '총', '개', '나라', '4만', '여', '유전자', '염색체', '수준', '규명', 'Nature', '20일', '밝히다', '2016.10', '20', '제공', '다양하다', '생물', '어떻다', '진화', '풀다', '낼', '수', '있다', '열쇠', '찾다', '것', '향후', '보이다', '발현', '조절', '쉬다', '체외수정', '번', '수백', '알', '얻다', '척추동물', '널리', '사용', '돼다', '2012년', '노벨', '생리', '의학상', '수상한', '존', '고든', 'Sir', 'John', 'Gordon', '이', '체세포', '핵치환', '실험', '통해', '어른', '세포', '다시', '배아', '걸', '처음', '보여주다', '그러나', '정작', '개구리', '다른', '동물', '어렵다', '인간', '부모', '로부터', '하나', '씩', '그룹', '물리다', '받다', '2', '배체', '가지', '두', '4', '가다', '때문', '분석', '복잡하다', '지난', '2010년', '서양', 'tropicalis', '를', '이용', '시도', '가지다', '서', '양', '비교', '배체수', '변화', '따르다', '차이', '방법', '조상', '약', '4천', '800만년', '전', '종', '분화', '1천', 

In [119]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_no_pretrained.wv.most_similar('흡연')

[('정확하다', 0.9990271329879761),
 ('자신', 0.9990252256393433),
 ('어떻다', 0.9989774227142334),
 ('자체', 0.9989314079284668),
 ('의미', 0.9989103078842163),
 ('독특하다', 0.9989053010940552),
 ('영향', 0.998896598815918),
 ('저하', 0.9988710284233093),
 ('소자', 0.9988604784011841),
 ('진단', 0.9988458156585693)]

In [120]:
model_no_pretrained.wv.most_similar('자동차')

[('그', 0.9997779130935669),
 ('인류', 0.9995789527893066),
 ('하나', 0.999565839767456),
 ('경우', 0.9995344281196594),
 ('변신', 0.9994568824768066),
 ('속', 0.9993412494659424),
 ('흥분', 0.999305248260498),
 ('계획', 0.9992903470993042),
 ('다음', 0.9992735385894775),
 ('일', 0.999258279800415)]

잘 학습되지는 않은 느낌이다.

In [121]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(size = pretrained.wv.vector_size, min_count = 0)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 4123


In [122]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 31727


In [123]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 15)

(269318, 273105)

In [124]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar('흡연')

[('대마초', 0.6287971138954163),
 ('도박', 0.5945263504981995),
 ('중독', 0.5854969024658203),
 ('음주', 0.5775356292724609),
 ('피임', 0.577086865901947),
 ('투약', 0.5655815601348877),
 ('복용', 0.5621966123580933),
 ('마약', 0.5592858791351318),
 ('매춘', 0.5585386753082275),
 ('수혈', 0.5553884506225586)]

In [125]:
model_pretrained.wv.most_similar('자동차')

[('승용차', 0.7440077662467957),
 ('이륜차', 0.6633291244506836),
 ('오토바이', 0.6568530201911926),
 ('자가용', 0.6534370183944702),
 ('스포츠카', 0.6464002132415771),
 ('차량', 0.6419311761856079),
 ('경주용', 0.6317964792251587),
 ('모터', 0.6312204599380493),
 ('미니밴', 0.6205192804336548),
 ('자전거', 0.618431031703949)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]

사전 학습 없이 모델을 학습한 경우와 그렇지 않은 경우 모델의 성능이 많이 차이가 났다.  
그런 차이가 발생한 이유에는 사전학습된 모델의 단어 수는 30185개인 것에 비해 기존의 단어 수는 4123개만 있었기 때문에 유의미한 연관성을 만들 수 있을만큼의 단어수가 충분하지 않아서 성능의 차이가 나타났던 것 같다. 
그리고 기존의 기사를 tokenize하는 과정에서 단어들이 잘 뭉쳐지지 않아서 그런 것도 있어 보인다.

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

시각화를 하니 너무 뭉쳐있어서 가장 거리가 가까운 단어를 찾는 과정을 하지 못했다.  
공대와 성균관대의 벡터가 가까웠다.  
의약-가계도-탈모의 벡터가 가까웠다.

In [126]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

4123

In [127]:
# embedding 결과 가져오기 (2차원에 할지 3차원에 할지는 내맘대로)
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_no_pretrained.wv[i])

In [137]:
pca = PCA(random_state = 0).fit_transform(vector)[:,:2]
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

In [138]:
print(pca.shape)
print(tsne.shape)

(4123, 2)
(4123, 2)


In [141]:
# 시각화해보세요!

data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    text = ['point #{}'.format(i) for i in word],
    mode='markers',
    marker=dict(
        size=3,

        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [133]:
# 3차원에도 시각화해보자
pca3 = PCA(random_state = 0).fit_transform(vector)[:,:3]

In [142]:
# 시각화해보세요!

data = go.Scatter3d(
    x=pca3[:,0],
    y=pca3[:,1],
    z=pca3[:,2],
    text = ['point #{}'.format(i) for i in word],
    mode='markers',
    marker=dict(
        size=3,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

In [143]:
# tsne

data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in word],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!  

위의 텍스트마이닝 과정 자체가 제대로 학습된 편은 아니어서 두 결과 중 뭐가 좋고 아니고는 판단을 할 수가 없다.  
그런데 perplexity를 줄일수록 전체적으로 점들이 모인 형태가 보여지는 것 같다.

In [144]:
# t-SNE
tsne = TSNE(n_components = 2,
            random_state = 0,
            perplexity = 38,
            learning_rate = 0.05,
            init = 'random',
            n_iter = 1000).fit_transform(vector)
# 시각화

# tsne

data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in word],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [145]:
# t-SNE
tsne = TSNE(n_components = 2,
            random_state = 0,
            perplexity = 20,
            learning_rate = 0.05,
            init = 'random',
            n_iter = 1000).fit_transform(vector)
# 시각화

# tsne

data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in word],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 수고하셨습니다!